[View in Colaboratory](https://colab.research.google.com/github/jrstevenjlab/ColaboratoryAtomicLab/blob/master/plot_skittles.ipynb)

The goal of the excercise is to fit the function 

$$S(x) = B + C (x - x_0)^{n}$$

to data for power as a function of distance that you recorded in lab.  The parameters of the function are:

*   B: a background term, the value the data would have at x=infinity,
*   C: a scale factor which multiplies $(x-x_0)^n$, which translates the units
 from $distance^n$ to radiated power.
*   $x_0$: maybe the filament wasn't exactly at x=0? If so, this term allows
 the fit to try and correct for that.
*   n: the power in the power law, which we expect to be -2

The function given in the lab manual is a limiting form of this more general function, where B=0 and $x_0$=0.  Here we'll treat those as parameters of the model to be determined by the fit.

By "fitting" the data we mean that the parameters of the model are allowed to vary until the curve defined by this function best match the data you've measured.  This is done by minimizing the $\chi^2$ which is a metric for comparing the model function to the observed data.  The $\chi^2$ is defined as

$$\chi^{2} = \sum_{i=0} \frac{S_i - S_{model}(x_i)}{\sigma_{S_{i}}}$$

where the sum of $i$ is over your observed data points
*  $S_i$ are the individual data points for the power (y-axis value), 
*  $\sigma_{S_{i}}$ are the errors on the individual data points,
*  and $S_{model}(x_i)$ is the value of your model function at the position $x_i$

The steps below describe the process that you should follow:

1.   Enter your data for power and position in the ordered lists below, replacing the example data already there.
2.   Run the fit code by pressing the "Play" button, which will cause a few things to happen:
> * Your data will be plotted as blue points in the figure below the code 
> * The `curve_fit` command in the code will find the "best fit" values for the paremeters of the model which gives the smallest value for the $\chi^2$ possible (ie. it "minimizes" the $\chi^2$).
> * The function S(x) with these "best fit" parameters will be plotted in red overlayed with your data points
> * The values for the parameters are output as text below the figure, along with the uncertainties on those parameters.

Finally, we need evaluate the "quality" of our fit by looking at the $\chi^{2}/DOF$, where DOF is the **Degrees Of Freedom** of the fit, defined as:

$$DOF = # data points - # parameters$$

In our case the # parameters = 4 (B, C, $x_0$, and n). and the # data points is given by the number of points you observed in lab.   In general, a "good" fit will have a $\chi^{2}/DOF$ close to 1.  
*  If $\chi^{2}/DOF$ >> 1, then the errors were likely **underestimated** meaning the model function does not describe the data.
*  If $\chi^{2}/DOF$ << 1, then the errors were likely **overestimated** meaning the model describes the data better than it should (statistically).





In [13]:
# import modules needed for data analysis
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# import pandas and excel importer
import pandas as pd
!pip install -q xlrd

# import modules for reading data from Google Sheets
!pip install --upgrade -q gspread

import io
from google.colab import auth
from google.colab import files
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# function for skittle volume vs group number with paramters a and b
def VolumeModel(x, a, b):
  #xtemp = x[0])
  return a * x + b

##########################
# LOAD YOUR DATA HERE! #
##########################

# Open our new sheet and read some data.
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1a_keMr8OFydGUFUiJ-j2mwzv-dmlv7RIlgB238As9bs/edit#gid=2092680776').sheet1

uploaded = files.upload()
df = pd.read_excel(io.StringIO(uploaded['SkittlesData.xlsx'].decode('utf-8')))

# Wednesday afternoon data
xw=df['Avg V'].tolist() #list(np.int_(worksheet.col_values(1)[1:])) # first column is group number (x-axis variable)
yw=list(np.float_(worksheet.col_values(2)[1:])) # second column is average volume (y-axis variable)
eyw=list(np.float_(worksheet.col_values(3)[1:])) # thrid column is averge volume std. dev. (y-axis error)

# Thursday morning data
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1a_keMr8OFydGUFUiJ-j2mwzv-dmlv7RIlgB238As9bs/edit#gid=2092680776').get_worksheet(1)
xt_morn=list(np.float_(worksheet.col_values(1)[1:])) # first column is group number (x-axis variable)
yt_morn=list(np.float_(worksheet.col_values(2)[1:])) # second column is average volume (y-axis variable)
eyt_morn=list(np.float_(worksheet.col_values(3)[1:])) # thrid column is averge volume std. dev. (y-axis error)

# Thursday morning data
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1a_keMr8OFydGUFUiJ-j2mwzv-dmlv7RIlgB238As9bs/edit#gid=2092680776').get_worksheet(2)
xt_aftn=list(np.float_(worksheet.col_values(1)[1:])) # first column is group number (x-axis variable)
yt_aftn=list(np.float_(worksheet.col_values(2)[1:])) # second column is average volume (y-axis variable)
eyt_aftn=list(np.float_(worksheet.col_values(3)[1:])) # thrid column is averge volume std. dev. (y-axis error)

# plot the data on a new figure
plt.figure() # create new figure for plotting
plt.errorbar(xw, yw, eyw, fmt='or') # plot volume vs group number, where eyw is the uncertainty on your volume values
plt.errorbar(xt_morn, yt_morn, eyt_morn, fmt='ok')
plt.errorbar(xt_aftn, yt_aftn, eyt_aftn, fmt='ob')
plt.axis([0, 10, 0, 1.6])  # set axes [xmin, xmax, Smin, Smax]
plt.xlabel('group number') # set x-axis label
plt.ylabel('skittle volume $\pm 1\sigma$ (cm$^3$)') # set y-axis label 

# fit the data
initialParameters = [0.75,0] # start with initial guess for fit parameters [a, b]
finalParameters, finalParameterErrors = curve_fit(VolumeModel, xw, yw, initialParameters, eyw, True) #
print(finalParameters)

# show red curve for fit results and plot on top of the data
print(type(xw[0].item()))
print(type(finalParameters[0].item()))
plt.plot(xw, VolumeModel(xw, finalParameters[0].item(), finalParameters[1].item())) #, 'r--', label='fit: a=%5.3f, b=%5.3f' % tuple(finalParameters))
plt.show()

# output fit parameter values and errors
print("Model parameters:")
print("a   =", finalParameters[0], " +/- ",np.sqrt(finalParameterErrors[0,0]))
print("b   =", finalParameters[1], " +/- ",np.sqrt(finalParameterErrors[1,1]))



Saving SkittlesData.xlsx to SkittlesData (2).xlsx


UnicodeDecodeError: ignored